# Getting Started

## Install `doitoml`

Use your python package manager of choice to install a release of `doitoml`:

````{tab-set}

```{tab-item} PyPI
~~~bash
pip install doitoml
~~~
```

```{tab-item} conda-forge
~~~bash
mamba install -c conda-forge doitoml
~~~

or

~~~bash
conda install -c conda-forge doitoml
~~~
```

````

<details>

<summary>This will install...</summary>

- `doitoml` and its dependencies
  - `doit[toml]`
  - `tomli` (if you're on python <3.11)

</details>

## Configure `doit`

Like many Python-based tools, `doit` prefers finding its [configuration in `pyproject.toml`](https://pydoit.org/configuration.html#pyproject-toml) under the `[tool.doit]` prefix. 

`doitoml` provide an _entry point_ for `doit.LOADER`, which offers task discovery features from files. As an alternative, advanced users of `dodo.py` can use the Python API.

```toml
# pyproject.toml
[tool.doit]
loader = "doitoml"
```

### Write Tasks

_Tasks_ are the most important part of using `doit`. These are written in the `[tool.doitoml.tasks]` prefix. For readability, it's encouraged to fully write out tasks in their own sections.

```toml
# pyproject.toml
[tool.doit]
loader = "doitoml"

[tool.doitoml.tasks.hello]
actions = ['echo "hello world"']
```

> **Note**
>
>While the above, cross-platform _shell_ task is simple and powerful, this pattern inroduces a number of risks around shell escaping. Keep reading for more about other kinds of tasks.

### Parts of tasks

Even in vanilla `doit`, there are a large number of configurable options. Few tasks will need close to all of them, but a few are worth pointing out:

| field | kind | description |
|-|-|-|
| `actions` | list of (strings or list of strings) | the raw shell command or shell tokens 
| `targets` | list of paths | files created or modified by a task
| `file_dep` | list of paths | files that, when changed, will cause a task to be run 
| `task_dep` | list of fully-qualified _task ids_ | the names (with optional `*` wildcards) of any _other_ tasks that need to have been run before a task
| `doc` | string | a text description of a task
| `meta` | dict | a dictionary of extra values, not used directly by `doit`
| `uptodate` | list of (string, bool or dict) | update checkers that complement `file_dep` and `task_dep`

See the [cheatsheet](../reference/cheatsheet.md) for more.

## Use `doit`

### Run Tasks

By default, `doit [run]` will run **all** known tasks.

```bash
doit
```

### List Tasks

Now you are ready to start working with your tasks:

```bash
doit list --all --status
```

This will print out the names of all runnable tasks, and whether they have been (or need to be) run.

### Inspect Tasks

With the name of the task from `list`, you can learn more about a specific task.

```bash
doit info hello
```

This prints out more information about the task.

## Extend with `doitoml`

Now that some tasks are running, `doitoml` provides some tools to help keep your task definitions short and reusable.

| section | description |
|-|-|
| [`env`](#env) | environment variables
| [`paths`](#paths) | reusable paths
| [`tokens`](#tokens) | command shell tokens
| [`templates`](../how-to/templates.md) | data-driven templates

### `paths`

Paths are the most robust way to manage the relationships between tasks. `doitoml.paths` allow for defining paths which can be reused in multiple tasks.

#### Simple paths

The simplest path is just a list of strings, treated as relative paths to the file that defines them.

```toml
[tool.doitoml.paths]
hello = ["hello.txt"]
hola = ["es/hola.txt"]
```

> **Note**
>
> Only POSIX-style slashes `/` are supported as `\` is used as an escape character by most syntaxes.

#### Reusing paths

Paths can be reused in _other_ paths, with the `::` syntax:

```toml
[tool.doitoml.paths]
hello = ["hello.py"]
greetings = ["::hello"]
```

And in _token_-type tasks:

```toml
[tool.doitoml.tasks.hello]
actions = [["python", "::hello"]]
```


> **Note**
>
> A number of other [path patterns](../how-to/dsl.md) are available, and can also be [extended](../understanding/extending.md) with new syntax.

#### Referencing `env`

Paths can also reference environment variables.

```toml
[tool.doitoml.env]
MY_NUMBER = "42"

[tool.doitoml.paths]
my_build = "my-project-${MY_NUMBER}.zip"
```

### `env`

Environment variables are a cross-platform, cross-language way to provide consistent data across many processes.

```toml
[tool.doitoml.env]
MY_DATA = "some-data"
MY_OTHER_DATA = "more-of-${MY_DATA}"
```

> **Note**
>
> Only the POSIX-style `${ENV_VAR}` syntax is supported: `$ENV_VAR` and `%ENV_VAR%` have no particular meaning.



### `tokens`

The `tokens` section is useful for defining frequently-reused strings, and share the same namespace as `paths`.


```toml
[tool.doitoml.tokens]
py = ["python3"]
py_c = ["::py", "-c"]
py_m = ["::py", "-m"]
```

These are useful for keeping task actions short.

```toml
[tool.doitoml.tasks.hello]
actions = [["::py_c", "print('hello')"]]
```

### Summary

Building declarative tasks for `doit` with `doitoml` makes it easier to automate simple, software-driven tasks in a predictable, reproducible way, but this barely scratches the surface of what is possible.


> **Hint**
> 
> For an overview, see the [cheatsheet](../reference/cheatsheet.md).


Additionally, see more in-depth guides:

> TODO
>
> - _Defining tasks in multiple files_
> - _Extending `doitoml`_
> - _Data-driven projects_